In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
counties

In [ ]:
# load the data
df = pd.read_csv("data/rgdp_county_allindustry.csv")
# converting GeoFIPS to string and fill with leading zeros to match the GitHUB data
df['GeoFIPS'] = df['GeoFIPS'].astype(str).str.zfill(5) 
df.head()

In [ ]:
# calculate the percentage change
df_pct_chg = df.copy()
df_pct_chg = df_pct_chg.sort_values(by=['GeoFIPS', 'year']).reset_index(drop=True)
df_pct_chg['pct_chg'] = df_pct_chg['value'].pct_change()*100
df_pct_chg = df_pct_chg[df_pct_chg[year] != 2001].reset_index(drop=True)
df_pct_chg.head()

In [ ]:
df_pct_chg2019 = df_pct_chg[df_pct_chg['year'] == 2019].reset_index(drop=True)

In [ ]:
fig = px.choropleth_mapbox(df_pct_chg2019, geojson=counties, locations='GeoFIPS', color='pct_chg',
                           color_continuous_scale="RdBu_r",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'pct_chg':'change in GDP (%)'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# Create a list of dictionaries, each representing a different year's data
data = []
for year in df_pct_chg['year'].unique():
    data.append(
        dict(
            args=['visible', [False]*len(df_pct_chg['year'].unique())],
            label=str(year),
            method='update'
        )
    )

# Update the visibility for each year
for i in range(len(data)):
    year = data[i]['label']
    data[i]['args'][1][i] = True  # Toggle i'th trace to "visible"

# Create the figure with a dropdown menu
fig = go.Figure()

for year in df_pct_chg['year'].unique():
    fig.add_trace(
        go.Choroplethmapbox(
            geojson=counties,
            locations=df_pct_chg[df_pct_chg['year'] == year]['GeoFIPS'],
            z=df_pct_chg[df_pct_chg['year'] == year]['pct_chg'],
            colorscale="Viridis",
            zmin=0,
            zmax=12,
            marker_opacity=0.5,
            marker_line_width=0,
        )
    )

fig.update_layout(
    updatemenus=[
        dict(
            buttons=list(data),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# Create a list of dictionaries, each representing a different year's data
MIN = np.nanpercentile(df_pct_chg["pct_chg"], 10)
MAX = np.nanpercentile(df_pct_chg["pct_chg"], 90)
data = []
for i, year in enumerate(df_pct_chg['year'].unique()):
    visible = [False]*len(df_pct_chg['year'].unique())
    visible[i] = True
    data.append(
        dict(
            args=[{'visible': visible}],
            label=str(year),
            method='update'
        )
    )

# Create the figure with a dropdown menu
fig = go.Figure()

for year in df_pct_chg['year'].unique():
    fig.add_trace(
        go.Choroplethmapbox(
            geojson=counties,
            locations=df_pct_chg[df_pct_chg['year'] == year]['GeoFIPS'],
            z=df_pct_chg[df_pct_chg['year'] == year]['pct_chg'],
            colorscale="Viridis",
            zmin=MIN,
            zmax=MAX,
            marker_opacity=0.5,
            marker_line_width=0,
            visible = True if i == 0 else False
        )
    )

fig.update_layout(
    updatemenus=[
        dict(
            buttons=list(data),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="right",
            y=1.1,
            yanchor="top"
        ),
    ],
    autosize=False,
    width=1000,
    height=800
)

fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()